## Implementation using TensorFlow

Previously having implemented this using FastAI, although not the best is still functional, I have decided to use TensorFlow to implement this again, to allow me to be more flexible and learn more about this framework. After a lot of learning I am now at a point in understanding where creating this model shouldnt be too difficult. This project will be split into 4 parts:
* Loading and preparing the data
* Creation and compilation of the model
* Training the model
* Creating the agent (this time not in the notebook!)

### Library imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import gridspec

# matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells

### Part 1 - loading and preperation of the data